In [5]:
import requests
from bs4 import BeautifulSoup

# 대상 URL
url = "https://onstar.jj.ac.kr/"

# 웹 페이지 요청
response = requests.get(url)
html = response.text

# BeautifulSoup 객체 생성
soup = BeautifulSoup(html, "html.parser")

# 특정 div 태그 가져오기 (예: class="target-class"인 div 찾기)
div_data = soup.find("div", class_="ChildFrame popup")  # 특정 클래스가 있는 div
# 또는 특정 ID
# div_data = soup.find("div", id="target-id")

# div 내부의 텍스트 추출
if div_data:
    print(div_data.text.strip())  # 필요하면 .text 대신 .get_text(strip=True) 사용
else:
    print("해당 div를 찾을 수 없습니다.")

해당 div를 찾을 수 없습니다.


In [7]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")

In [11]:
import json
import os
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# JSON 파일 로드
json_file = "programs.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# 문서를 LangChain에서 사용할 형식으로 변환
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)
    documents.append(text)

# OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

# 검색용 Retriever 생성
retriever = vectorstore.as_retriever()

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.